In [3]:
import fastf1
import pandas as pd
from fastf1.req import RateLimitExceededError
import time
import logging
import concurrent.futures
from threading import Lock

def process_session_data(args):
    year, event, session_name = args
    try:
        # Load the session
        session = fastf1.get_session(year, event.RoundNumber, session_name)
        session.load()
        
        # Get the laps data
        laps_df = session.laps.copy()
        
        # Add metadata to laps
        laps_df['Year'] = year
        laps_df['EventName'] = event.EventName
        laps_df['SessionName'] = session_name
        laps_df['EventFormat'] = event.EventFormat
        laps_df['RoundNumber'] = event.RoundNumber
        
        # Get weather data
        weather_df = session.weather_data.copy()
        
        # Add metadata to weather
        weather_df['Year'] = year
        weather_df['EventName'] = event.EventName
        weather_df['SessionName'] = session_name
        weather_df['EventFormat'] = event.EventFormat
        weather_df['RoundNumber'] = event.RoundNumber
        
        return {'laps': laps_df, 'weather': weather_df}
        
    except Exception as e:
        print(f"Error processing {session_name} for {event.EventName} in {year}: {e}")
        return None

def collect_f1_data(start_year=2018, end_year=2024, max_workers=3):
    # Enable cache
    fastf1.Cache.enable_cache('cache')
    fastf1.set_log_level(logging.DEBUG)
    
    # Initialize lists and synchronization objects
    lap_dataframes = []
    weather_dataframes = []
    df_lock = Lock()
    save_counter = 0
    counter_lock = Lock()
    
    def save_progress():
        if lap_dataframes and weather_dataframes:
            # Save laps progress
            temp_laps_df = pd.concat(lap_dataframes, ignore_index=True)
            temp_laps_df.to_csv('lap_data_progress.csv', index=False)
            
            # Save weather progress
            temp_weather_df = pd.concat(weather_dataframes, ignore_index=True)
            temp_weather_df.to_csv('weather_data_progress.csv', index=False)
            
            print("Progress saved!")
    
    def process_and_save(future):
        nonlocal save_counter
        result = future.result()
        if result is not None:
            with df_lock:
                lap_dataframes.append(result['laps'])
                weather_dataframes.append(result['weather'])
                with counter_lock:
                    save_counter += 1
                    if save_counter % 5 == 0:
                        save_progress()
    
    # Generate all tasks
    tasks = []
    for year in range(start_year, end_year):
        print(f"\nProcessing year {year}")
        schedule = fastf1.get_event_schedule(year)
        
        for event in schedule.itertuples():
            print(f"\nQueuing event: {event.EventName}")
            
            # Get all session types based on event format
            if event.EventFormat == 'conventional':
                sessions = ['FP1', 'FP2', 'FP3', 'Q', 'R']
            elif event.EventFormat == 'sprint':
                sessions = ['FP1', 'Q', 'FP2', 'S', 'R']
            elif event.EventFormat == 'sprint_shootout':
                sessions = ['FP1', 'Q', 'SS', 'S', 'R']
            elif event.EventFormat == 'sprint_qualifying':
                sessions = ['FP1', 'SQ', 'S', 'Q', 'R']
            else:
                sessions = ['FP1', 'FP2', 'FP3']
            
            for session_name in sessions:
                tasks.append((year, event, session_name))
    
    try:
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all tasks and add callbacks
            futures = []
            for task in tasks:
                future = executor.submit(process_session_data, task)
                future.add_done_callback(process_and_save)
                futures.append(future)
                time.sleep(0.5)  # Small delay between submissions to prevent overwhelming the API
            
            # Wait for all tasks to complete
            concurrent.futures.wait(futures)
            
    except KeyboardInterrupt:
        print("\nScript interrupted by user. Saving progress...")
    
    finally:
        if lap_dataframes and weather_dataframes:
            # Save all lap data
            all_lap_data = pd.concat(lap_dataframes, ignore_index=True)
            all_lap_data.to_csv('all_lap_data_final.csv', index=False)
            
            # Save all weather data
            all_weather_data = pd.concat(weather_dataframes, ignore_index=True)
            all_weather_data.to_csv('all_weather_data_final.csv', index=False)
            
            print("\nData collection complete. Data saved to:")
            print("- 'all_lap_data_final.csv'")
            print("- 'all_weather_data_final.csv'")
            return {'laps': all_lap_data, 'weather': all_weather_data}
        else:
            print("No data was collected!")
            return None

# Run the collection
if __name__ == "__main__":
    data = collect_f1_data(start_year=2018, end_year=2024, max_workers=3)

core           INFO 	Loading data for Australian Grand Prix - Practice 1 [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...



Processing year 2018

Queuing event: Australian Grand Prix

Queuing event: Bahrain Grand Prix

Queuing event: Chinese Grand Prix

Queuing event: Azerbaijan Grand Prix

Queuing event: Spanish Grand Prix

Queuing event: Monaco Grand Prix

Queuing event: Canadian Grand Prix

Queuing event: French Grand Prix

Queuing event: Austrian Grand Prix

Queuing event: British Grand Prix

Queuing event: German Grand Prix

Queuing event: Hungarian Grand Prix

Queuing event: Belgian Grand Prix

Queuing event: Italian Grand Prix

Queuing event: Singapore Grand Prix

Queuing event: Russian Grand Prix

Queuing event: Japanese Grand Prix

Queuing event: United States Grand Prix

Queuing event: Mexican Grand Prix

Queuing event: Brazilian Grand Prix

Queuing event: Abu Dhabi Grand Prix

Processing year 2019

Queuing event: Australian Grand Prix

Queuing event: Bahrain Grand Prix

Queuing event: Chinese Grand Prix

Queuing event: Azerbaijan Grand Prix

Queuing event: Spanish Grand Prix

Queuing event: Mona

req            INFO 	Using cached data for car_data
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
core           INFO 	Loading data for Australian Grand Prix - Practice 2 [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver 11: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 14: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Driver 18: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver  2: Lap timing integrity check failed for 1 lap(s)
core        WARNING 

Progress saved!


core        WARNING 	Driver 55: Lap timing integrity check failed for 1 lap(s)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['10', '11', '14', '16', '18', '2', '20', '27', '28', '3', '31', '33', '35', '44', '5', '55', '7', '77', '8', '9']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing 

...

logger      WARNING 	Failed to load schedule from FastF1 backend!
logger      WARNING 	Failed to load schedule from FastF1 backend!
logger      WARNING 	Failed to load schedule from FastF1 backend!
logger        DEBUG 	Traceback for failure in FastF1 schedule
Traceback (most recent call last):
  File "/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/fastf1/logger.py", line 151, in __wrapped
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/fastf1/events.py", line 584, in _get_schedule_ff1
    response = Cache.requests_get(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/fastf1/req.py", line 303, in requests_get
    return cls._cached_request('GET', url, **kwa

Error processing FP1 for Austrian Grand Prix in 2020: The data you are trying to access has not been loaded yet. See `Session.load`


logger      WARNING 	Failed to load race control messages!
logger        DEBUG 	Traceback for failure in race control messages
Traceback (most recent call last):
  File "/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/fastf1/logger.py", line 151, in __wrapped
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/fastf1/core.py", line 2399, in _load_race_control_messages
    race_control_messages = api.race_control_messages(self.api_path,
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/fastf1/req.py", line 479, in _cached_api_request
    data = func(api_path, **func_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/

Error processing FP3 for Austrian Grand Prix in 2020: The data you are trying to access has not been loaded yet. See `Session.load`
Error processing FP2 for Austrian Grand Prix in 2020: The data you are trying to access has not been loaded yet. See `Session.load`


logger      WARNING 	Failed to load schedule from FastF1 backend!
logger        DEBUG 	Traceback for failure in FastF1 schedule
Traceback (most recent call last):
  File "/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/fastf1/logger.py", line 151, in __wrapped
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/fastf1/events.py", line 584, in _get_schedule_ff1
    response = Cache.requests_get(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/fastf1/req.py", line 303, in requests_get
    return cls._cached_request('GET', url, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mini